<a href="https://colab.research.google.com/github/farhanwadia/nlp_g14/blob/Geet/p3_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up notebook


In [3]:
# cloning our Project Repo
!git clone -b Geet https://github.com/farhanwadia/nlp_g14.git

Cloning into 'nlp_g14'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 6), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), 1.17 MiB | 8.10 MiB/s, done.


In [4]:
%cd nlp_g14/

/content/nlp_g14


In [5]:
! ls

MIE1624Project.ipynb	reddit_scraper.ipynb
p3_TopicModeling.ipynb	russia_ukraine_reddit_comments.csv
README.md


# Import Libraries


In [17]:
# importing basic python modules that we most probably will be using in the notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
try: 
  import langdetect
except:
  !pip install langdetect
  import langdetect

# to make sure our plots are inline with the code cells
%matplotlib inline 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=662ba644d0b61137fa2cddbc7b7f1c3c8e7ab5bec3276e9a2c42406e96d0d107
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [7]:
reddit_data = 'russia_ukraine_reddit_comments.csv'
reddit_df = pd.read_csv(reddit_data)
reddit_df.head()

,Post - Author,Post - Date,Post - Is Distinguished,Post - Is Edited,Post - Is Original Content,Post - Is Locked,Post - Name,Post - num_comments,Post - over_18,Post - Permalink,...,Author,Date,Parent ID Prefix,Parent ID,Is Distinguished,Is Edited,Is Stickied,Permalink,Score,Body
0,flyingcatwithhorns,2023-03-15 14:16:41,NaN,False,False,False,t3_11rxpnv,392,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,...,iwsfutcmd,2023-03-15 19:15:22,t3,11rxpnv,NaN,False,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,75,Myanmar?\n\nwell that's a surprise
1,flyingcatwithhorns,2023-03-15 14:16:41,NaN,False,False,False,t3_11rxpnv,392,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,...,snowday784,2023-03-15 17:25:13,t3,11rxpnv,NaN,False,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,316,Bolivia what is you doing bby
2,flyingcatwithhorns,2023-03-15 14:16:41,NaN,False,False,False,t3_11rxpnv,392,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,...,micahsaurus,2023-03-15 15:32:45,t3,11rxpnv,NaN,False,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,202,"Kind of misleading.\n\nIt should read, “Which ..."
3,flyingcatwithhorns,2023-03-15 14:16:41,NaN,False,False,False,t3_11rxpnv,392,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,...,grisioco,2023-03-15 14:28:30,t3,11rxpnv,NaN,False,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,371,you know youre in the right when the only coun...
4,flyingcatwithhorns,2023-03-15 14:16:41,NaN,False,False,False,t3_11rxpnv,392,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,...,Foreign_Phone59,2023-03-15 22:18:48,t3,11rxpnv,NaN,False,False,http://www.reddit.com/r/MapPorn/comments/11rxp...,50,Not Afganistan and Myanmar being beacons of re...


In [10]:
reddit_df['Date'].value_counts()

2022-03-03 09:57:35    2
2022-03-01 17:12:20    2
2023-02-20 19:49:39    2
2022-02-24 04:43:07    2
2022-03-01 17:45:34    2
                      ..
2023-02-24 15:57:55    1
2023-02-24 20:08:23    1
2023-02-24 18:33:48    1
2023-02-24 17:59:51    1
2023-02-21 17:36:10    1
Name: Date, Length: 7806, dtype: int64

In [12]:
red_text_df = reddit_df[['Body', 'Date']].copy(deep=True)

In [14]:
# Check nulls
red_text_df.isnull().sum(axis=0)

Body    0
Date    0
dtype: int64

In [16]:
# Drop Duplicates
print(red_text_df.info())
red_text_df.drop_duplicates(subset='Body', inplace=True)
red_text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7852 entries, 0 to 7851
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    7852 non-null   object
 1   Date    7852 non-null   object
dtypes: object(2)
memory usage: 122.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7645 entries, 0 to 7851
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    7645 non-null   object
 1   Date    7645 non-null   object
dtypes: object(2)
memory usage: 179.2+ KB


In [18]:
# Filter to only have english

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

def detect_language(text):
  try:
    return detect(text)
  except:
    return None

red_text_df['language'] = red_text_df['Body'].apply(detect_language)
red_text_df.head()

,Body,Date,language
0,Myanmar?\n\nwell that's a surprise,2023-03-15 19:15:22,en
1,Bolivia what is you doing bby,2023-03-15 17:25:13,en
2,"Kind of misleading.\n\nIt should read, “Which ...",2023-03-15 15:32:45,en
3,you know youre in the right when the only coun...,2023-03-15 14:28:30,en
4,Not Afganistan and Myanmar being beacons of re...,2023-03-15 22:18:48,en
